## Backpropagation
### LSTM
$
% Forget-Gate
\newcommand{\F}{\color{orange}{f_t}}
\newcommand{\Ffull}[1][]{\color{orange}{\sigma^{#1} \left( W_f + U_f \cdot h_{t-1} \right)}}
\newcommand{\f}[1]{\color{orange}{f^t_{#1}}}
\newcommand{\ffull}[2][]{\color{orange}{\sigma^{#1} \left(w^f_{#2} x_t + u^f_{#2 1} h^{t-1}_1 + u^f_{#2 2}h^{t-1}_2 \right)}}
\newcommand{\Wf}{\color{orange}{W_f}}
\newcommand{\Uf}{\color{orange}{U_f}}
\newcommand{\wf}[1]{\color{orange}{w_{#1}^f}}
\newcommand{\uf}[2]{\color{orange}{u_{#1 #2}^f}}
% Input-Gate
\newcommand{\I}{\color{red}{i_t}}
\newcommand{\Ifull}[1][]{\color{red}{\sigma^{#1} \left( W_i + U_i \cdot h_{t-1} \right)}}
\newcommand{\i}[1]{\color{red}{i^t_{#1}}}
\newcommand{\ifull}[2][]{\color{red}{\sigma^{#1} \left( w^i_{#2} x_t + u^i_{#2 1} h^{t-1}_1 + u^i_{#2 2} h^{t-1}_2 \right)}}
\newcommand{\Wi}{\color{red}{W_i}}
\newcommand{\Ui}{\color{red}{U_i}}
\newcommand{\wi}[1]{\color{red}{w_{#1}^i}}
\newcommand{\ui}[2]{\color{red}{u_{#1 #2}^i}}
% Output-Gate
\newcommand{\O}{\color{blue}{o_t}}
\newcommand{\Ofull}[1][]{\color{blue}{\sigma^{#1} \left( W_o + U_o \cdot h_{t-1} \right)}}
\newcommand{\o}[1]{\color{blue}{o^t_{#1}}}
\newcommand{\ofull}[2][]{\color{blue}{\sigma^{#1} \left( w^o_{#2} x_t + u^o_{#2 1} h^{t-1}_1 + u^o_{#2 2} h^{t-1}_2 \right)}}
\newcommand{\Wo}{\color{blue}{W_o}}
\newcommand{\Uo}{\color{blue}{U_o}}
\newcommand{\wo}[1]{\color{blue}{w_{#1}^o}}
\newcommand{\uo}[2]{\color{blue}{u_{#1 #2}^o}}
% Cell (c without tilde - or however the fuck it's supposed to be called)
\newcommand{\C}[1][]{\color{lime}{c_{t #1}}}
\newcommand{\Cfull}{\F * \C[-1] + \I * \Ctil}
\newcommand{\c}[2][]{\color{lime}{c^{t #1}_{#2}}}
% Candidate (c with tilde - again: whatever, it's fine.)
\newcommand{\Ctil}{\color{green}{\tilde{c}_t}}
\newcommand{\Ctilfull}[1][]{\color{green}{g^{#1} \left( W_c x_t + U_c \cdot h_{t-1} \right)}}
\newcommand{\ctil}[2][]{\color{green}{g^{#1} \left( w^c_{#2} x_t + u^c_{#2 1} h^{t-1}_1 + u^c_{#2 2} h^{t-1}_2 \right)}}
\newcommand{\ctils}[1]{\color{green}{\tilde{c}_{#1}}}
\newcommand{\Wc}{\color{green}{W_c}}
\newcommand{\Uc}{\color{green}{U_c}}
\newcommand{\wc}[1]{\color{green}{w_{#1}^c}}
\newcommand{\uc}[2]{\color{green}{u_{#1 #2}^c}}
% Miscellaneous
\newcommand{\yhat}{\hat{y}_t}
\newcommand{\error}{\left(y - \yhat \right)}
\newcommand{\deriv}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\dxt}[1]{\color{#1}{x_t}}
\DeclareMathOperator{\diag}{diag}
\newcommand{\myH}[2][-1]{\color{#2}{h_{t #1}}}
\newcommand{\myh}[2][red]{\color{#1}{h^{t-1}_{#2}}}
$
So the next step is to take the neatly separated equations $(1)$ to $(6)$ and plug them all in.

\begin{equation}
\begin{split}
\yhat &= w^T \cdot \myH[]{red}
      = w^T \cdot \left[ \O * g(\C) \right] \\
      &= w^T \left[ \Ofull * g(\Cfull) \right] \\
      &= w^T \cdot \Bigg[ \begin{pmatrix} \ofull{1} \\ \ofull{2} \end{pmatrix} \\ 
      &\quad\quad* g 
                         \begin{pmatrix} 
                             \ffull{1} \c[-1]{1} + \ifull{1} + \ctil{1} \\ 
                             \ffull{2} \c[-1]{2} + \ifull{2} + \ctil{2}
                         \end{pmatrix} \Bigg]
\end{split}
\end{equation}

which, if written in its single equation form, gives

\begin{alignat}{2}
\yhat &= w_1 \ofull{1} g\left(\ffull{1} \c[-1]{1} + \\ \ifull{1} \ctil{1}\right)\\ 
      &+ w_2 \ofull{2} g\left(\ffull{2} \c[-1]{2} + \\ \ifull{2} \ctil{2}\right) \tag{7}
\end{alignat}

Now this is arguably neither neat, nor pretty. However, since we are looking at a very low dimensional example it is a good opportunity to look into the backpropagation algorithm using high-school math only. And we can do that in a straightforward manner looking at equation $(7)$.

In this section we are going to take the partial derivative of $(7)$ with respect to each weight and using these results we will piece the gradients together. Moreover, once we have done that we will try to find a set of matrix operations, that will result in the gradients we previously puzzled together.

#### Weights of the output gate

Let's start with the weights of the output gate. If we take the partial derivatives with respect to $\wo{1}$ and $\wo{2}$ respectively we obtain:

\begin{equation}
\deriv{\yhat}{\wo{1}} = w_1 g(\c{1}) \ofull[\prime]{1} \ctil{1} \dxt{blue}
\end{equation}

\begin{equation}
\deriv{\yhat}{\wo{2}} = w_2 g(\c{2}) \ofull[\prime]{2} \ctil{2} \dxt{blue}
\end{equation}

Which we now only need to combine in order to get the gradient of $\yhat$ with respect to $\Wo$.

\begin{align}
\deriv{\yhat}{\Wo} &= \begin{bmatrix} \deriv{\yhat}{\wo{1}} \\ \deriv{\yhat}{\wo{2}} \end{bmatrix} \\
                   &= w * g(\C) * \Ofull[\prime] * \Ctil\ \dxt{blue} \\
                   &= \diag(w) \cdot \diag(g(\C)) \cdot \diag(\Ofull[\prime]) \cdot \Ctil \ \dxt{blue}
\end{align}

The last line is just a way of describing the same formula, without the use of element-wise multiplication, relying just on scalar and dot products. However, while this satisfies more conventional algebraic notational standards it is computationally more efficient to implement the gradients using Hadamard products.

Similarly, we obtain the partial derivatives for each element of $\Uo$.

\begin{equation}
\deriv{\yhat}{\uo{1}{1}} = w_1 g(\c{1}) \ofull[\prime]{1} \ctil{1} \myh[blue]{1}
\end{equation}

\begin{equation}
\deriv{\yhat}{\uo{1}{2}} = w_1 g(\c{1}) \ofull[\prime]{1} \ctil{1} \myh[blue]{2}
\end{equation}

\begin{equation}
\deriv{\yhat}{\uo{2}{1}} = w_2 g(\c{2}) \ofull[\prime]{2} \ctil{2} \myh[blue]{1}
\end{equation}

\begin{equation}
\deriv{\yhat}{\uo{2}{2}} = w_2 g(\c{2}) \ofull[\prime]{2} \ctil{2} \myh[blue]{2}
\end{equation}

And by combining them into the jacobian of $\yhat$ with respect to $\Uo$ we get

\begin{align}
\deriv{\yhat}{\Uo} &= \begin{bmatrix}
                         \deriv{\yhat}{\uo{1}{1}} & \deriv{\yhat}{\uo{1}{2}} \\
                         \deriv{\yhat}{\uo{2}{1}} & \deriv{\yhat}{\uo{2}{2}}
                     \end{bmatrix} \\
                   &= \begin{bmatrix}
                         w_1 g(\c{1}) \ofull[\prime]{1} \ctils{1} \myh[blue]{1} &
                         w_1 g(\c{1}) \ofull[\prime]{1} \ctils{1} \myh[blue]{2} \\
                         w_2 g(\c{2}) \ofull[\prime]{2} \ctils{2} \myh[blue]{1} &
                         w_2 g(\c{2}) \ofull[\prime]{2} \ctils{2} \myh[blue]{2} \\
                     \end{bmatrix}  \\
                  &= \begin{bmatrix} w & w \end{bmatrix} *
                     \begin{bmatrix} g(\C) & g(\C) \end{bmatrix} *
                     \begin{bmatrix} \Ofull[\prime] & \Ofull[\prime] \end{bmatrix} *
                     \begin{bmatrix} \Ctil & \Ctil \end{bmatrix} *
                     \begin{bmatrix} \myH{blue}^T \\ \myH{blue}^T \end{bmatrix} 
                     \\
                  &= \diag(w) \cdot \diag(g(\C)) \cdot \diag(\Ofull[\prime]) \cdot \diag(\Ctil) \cdot \mathbb{1} \mathbb{1}^T \cdot \diag(\myH{blue}).
\end{align}

#### Weights of the forget gate

By the same token we obtain the partial derivatives of $\yhat$ with respect to the elements of $\Wf$

\begin{equation}
\deriv{\yhat}{\wf{1}} = w_1 \o{1} g^\prime(\c{1}) \ffull[\prime]{1} \c[-1]{1} \dxt{orange}
\end{equation}

\begin{equation}
\deriv{\yhat}{\wf{2}} = w_2 \o{2} g^\prime(\c{2}) \ffull[\prime]{2} \c[-1]{2} \dxt{orange}
\end{equation}

and combine them to retrieve the gradient

\begin{align}
\deriv{\yhat}{\Wf} &= w * \O * g^\prime(\C) * \Ffull[\prime] * \C[-1] \ \dxt{orange} \\
                   &= \diag(w) \cdot \diag(\O) \cdot \diag\left(g^\prime(\C)\right) \cdot \diag(\Ffull[\prime]) \cdot \C[-1] \ \dxt{orange}.
\end{align}


For the partial derivatives of $\yhat$ with respect to the elements of $\Uf$ we obtain:

\begin{equation}
\deriv{\yhat}{\uf{1}{1}} = w_1 g^\prime(\c{1}) \ffull[\prime]{1} \c[-1]{1} \myh[orange]{1}
\end{equation}

\begin{equation}
\deriv{\yhat}{\uf{1}{2}} = w_1 g^\prime(\c{1}) \ffull[\prime]{1} \c[-1]{1} \myh[orange]{2}
\end{equation}

\begin{equation}
\deriv{\yhat}{\uf{2}{1}} = w_2 g^\prime(\c{2}) \ffull[\prime]{2} \c[-1]{2} \myh[orange]{1}
\end{equation}

\begin{equation}
\deriv{\yhat}{\uf{2}{2}} = w_2 g^\prime(\c{2}) \ffull[\prime]{2} \c[-1]{2} \myh[orange]{2}.
\end{equation}

Which we can combine in order to acquire the jacobian of $\yhat$ with respect to $\Uf$.

\begin{align}
\deriv{\yhat}{\Uf} &= \begin{bmatrix}
                         \deriv{\yhat}{\uf{1}{1}} & \deriv{\yhat}{\uf{1}{2}} \\
                         \deriv{\yhat}{\uf{2}{1}} & \deriv{\yhat}{\uf{2}{2}}
                     \end{bmatrix}
                   = \begin{bmatrix}
                          w_1 g^\prime(\c{1}) \ffull[\prime]{1} \c[-1]{1} \myh[orange]{1} &
                          w_1 g^\prime(\c{1}) \ffull[\prime]{1} \c[-1]{1} \myh[orange]{2} \\
                          w_2 g^\prime(\c{2}) \ffull[\prime]{2} \c[-1]{2} \myh[orange]{1} &
                          w_2 g^\prime(\c{2}) \ffull[\prime]{2} \c[-1]{2} \myh[orange]{2}
                     \end{bmatrix}
                  \\
                  &= \begin{bmatrix} w & w \end{bmatrix} *
                     \begin{bmatrix} \O & \O \end{bmatrix} *
                     g^\prime( \begin{bmatrix} \C & \C \end{bmatrix}) *
                     \begin{bmatrix} \Ffull[\prime] & \Ffull[\prime] \end{bmatrix} *
                     \begin{bmatrix} \C[-1] & \C[-1] \end{bmatrix} *
                     \begin{bmatrix} \myH{orange}^T \\ \myH{orange}^T \end{bmatrix} \\
                  &= \diag(w) \cdot \diag(\O) \cdot \diag\left(g^\prime(\C)\right) \cdot \diag(\Ffull[\prime]) \cdot \diag(\C[-1]) \cdot \mathbb{1} \mathbb{1}^T \cdot \diag(\myH{orange})
\end{align}


#### Weights of the input gate

Looking at the weights of the input gate we can proceed in the familiar fashion. We procure the partial derivatives of $\yhat$ with respect to $\wi{1}$ and $\wi{2}$

\begin{equation}
\deriv{\yhat}{\wi{1}} = w_1 \o{1} g^\prime(\c{1}) \ctil{1} \ifull[\prime]{1} \dxt{red},
\end{equation}


\begin{equation}
\deriv{\yhat}{\wi{2}} = w_2 \o{2} g^\prime(\c{2}) \ctil{2} \ifull[\prime]{2} \dxt{red}
\end{equation}

and proceed by collecting them in order to form the gradient of $\yhat$ with respect to $\Wi$

\begin{align}
\deriv{\yhat}{\Ui} &= w * \O * g^\prime(\C) * \Ctil * \Ifull[\prime] \dxt{red} \\
                   &= \diag(w) \cdot \diag(\O) \cdot \diag(g^\prime(\C)) \cdot \diag(\Ctil) \cdot \Ifull[\prime] \dxt{red}.
\end{align}


Equivalently we proceed with the partial derivatives of $\yhat$ with respect to the elements of $\Ui$, which are

\begin{equation}
\deriv{\yhat}{\ui{1}{1}} = w_1 \o{1} g^\prime(\c{1}) \ctil{1} \ifull[\prime]{1} \myh[red]{1},
\end{equation}

\begin{equation}
\deriv{\yhat}{\ui{1}{2}} = w_1 \o{1} g^\prime(\c{1}) \ctil{1} \ifull[\prime]{1} \myh[red]{2},
\end{equation}

\begin{equation}
\deriv{\yhat}{\ui{2}{1}} = w_2 \o{2} g^\prime(\c{2}) \ctil{2} \ifull[\prime]{2} \myh[red]{1},
\end{equation}
and
\begin{equation}
\deriv{\yhat}{\ui{2}{2}} = w_2 \o{2} g^\prime(\c{2}) \ctil{2} \ifull[\prime]{2} \myh[red]{2}.
\end{equation}

By collecting them into a matrix we obtain the jacobian of $\yhat$ with respect to $\Ui$

\begin{align}
\deriv{\yhat}{\Ui} &= \begin{bmatrix}
                          \deriv{\yhat}{\ui{1}{1}} & \deriv{\yhat}{\ui{1}{2}} \\
                          \deriv{\yhat}{\ui{2}{1}} & \deriv{\yhat}{\ui{2}{2}} 
                      \end{bmatrix} \\ 
                   &= \begin{bmatrix}
                          w_1 \o{1} g^\prime(\c{1}) \ctils{1} \ifull[\prime]{1} \myh[red]{1} &
                          w_1 \o{1} g^\prime(\c{1}) \ctils{1} \ifull[\prime]{1} \myh[red]{2} \\
                          w_2 \o{2} g^\prime(\c{2}) \ctils{2} \ifull[\prime]{2} \myh[red]{1} &
                          w_2 \o{2} g^\prime(\c{2}) \ctils{2} \ifull[\prime]{2} \myh[red]{2} \\
                      \end{bmatrix} \\
                   &= \begin{bmatrix} w & w \end{bmatrix} *
                      \begin{bmatrix} \O & \O \end{bmatrix} *
                      g^\prime\left(\begin{bmatrix} \C & \C \end{bmatrix}\right) *
                      \begin{bmatrix} \Ctil & \Ctil \end{bmatrix} *
                      \begin{bmatrix} \Ifull[\prime] & \Ifull[\prime] \end{bmatrix} *
                      \begin{bmatrix} \myH{red}^T \\ \myH{red}^T \end{bmatrix} \\
                   &= \diag(w) \cdot \diag(\O) \cdot \diag\left(g^\prime(\C)\right) \cdot \diag(\Ctil) \cdot \diag(\Ifull[\prime]) \cdot \mathbb{1} \mathbb{1}^T \cdot \diag(\myH{red})
\end{align}


### Weights of the candidate update

Again we begin by taking partial derivatives of $\yhat$ with respect to $\Wc$'s elements, such that we get

\begin{align}
\deriv{\yhat}{\wc{1}} = w_1 \o{1} g^\prime(\c{1}) \ifull{1} \ctil[\prime]{1} \dxt{green}
\end{align}

and

\begin{align}
\deriv{\yhat}{\wc{2}} = w_2 \o{2} g^\prime(\c{2}) \ifull{2} \ctil[\prime]{2} \dxt{green}.
\end{align}

Arranging the partial derivatives, we obtain the gradient of $\yhat$ with respect to $\Wc$

\begin{align}
\deriv{\yhat}{\Wc} = w * \O * g^\prime(\C) * \I * \Ctilfull[\prime]\ \dxt{green},
\end{align}

Similarly, we start by obtaining the partial derivatives of $\yhat$ with respect to each of $\Wc$'s elements.

\begin{equation}
\deriv{\yhat}{\uc{1}{1}} = w_1 \o{1} g^\prime(\c{1}) \i{1} \ctil[\prime]{1} \myh[green]{1},
\end{equation}

\begin{equation}
\deriv{\yhat}{\uc{1}{2}} = w_1 \o{1} g^\prime(\c{1}) \i{1} \ctil[\prime]{1} \myh[green]{2},
\end{equation}

\begin{equation}
\deriv{\yhat}{\uc{2}{1}} = w_2 \o{2} g^\prime(\c{2}) \i{2} \ctil[\prime]{2} \myh[green]{1}
\end{equation}
and
\begin{equation}
\deriv{\yhat}{\uc{2}{2}} = w_2 \o{2} g^\prime(\c{2}) \i{2} \ctil[\prime]{2} \myh[green]{2}.
\end{equation}

And by organizing these partial derivatives, we can form the jacobian of $\yhat$ with respect to $\Uc$

\begin{align}
\deriv{\yhat}{\Uc} &= \begin{bmatrix} 
                          \deriv{\yhat}{\uc{1}{1}} & \deriv{\yhat}{\uc{1}{2}} \\
                          \deriv{\yhat}{\uc{2}{1}} & \deriv{\yhat}{\uc{2}{2}}
                      \end{bmatrix} = 
                      \begin{bmatrix}
                          w_1 \o{1} g^\prime(\c{1}) \i{1} \ctil[\prime]{1} \myh[green]{1} &
                          w_1 \o{1} g^\prime(\c{1}) \i{1} \ctil[\prime]{1} \myh[green]{2} \\
                          w_2 \o{2} g^\prime(\c{2}) \i{2} \ctil[\prime]{2} \myh[green]{1} &
                          w_2 \o{2} g^\prime(\c{2}) \i{2} \ctil[\prime]{2} \myh[green]{2} 
                      \end{bmatrix} \\
                   &= \begin{bmatrix} w & w \end{bmatrix} *
                      \begin{bmatrix} \O & \O \end{bmatrix} *
                      g^\prime(\begin{bmatrix} \C & \C \end{bmatrix}) *
                      \begin{bmatrix} \I & \I \end{bmatrix} *
                      \begin{bmatrix} \Ctilfull[\prime] & \Ctilfull[\prime] \end{bmatrix} *
                      \begin{bmatrix} \myH{green}^T \\ \myH{green}^T \end{bmatrix} \\
                   &= \diag(w) \cdot 
                      \diag(\O) \cdot 
                      \diag\left(g^\prime(\C)\right) \cdot 
                      \diag(\I) \cdot 
                      \diag(\Ctilfull[\prime]) \cdot 
                      \mathbb{1} \mathbb{1}^T \cdot 
                      \diag(\myH{green})
\end{align}

## GRU
$
% Vector shortcuts
\newcommand{\Htil}{\color{green}{\tilde{h}_t}}
\newcommand{\H}{\color{green}{h_{t-1}}}
\newcommand{\R}{\color{blue}{r_t}}
\newcommand{\Z}{\color{red}{z_t}}
% Gates Matrix Notation
\newcommand{\Gfull}[1][]{\color{green}{g^{#1}\left( W_h x_t + U_h \cdot ( \R * \H)\right)}}
\newcommand{\Zfull}[1][]{\color{red}{\sigma^{#1} \left( W_z x_t + U_z \cdot h_{t-1} \right)}}
\newcommand{\Rfull}[1][]{\color{blue}{\sigma^{#1} \left( W_r x_t + U_r \cdot h_{t-1} \right)}}
% Vector Elements
\newcommand{\htil}[1]{\color{green}{\tilde{h}^t_{#1}}}
\newcommand{\h}[1]{\color{green}{h_{#1}^{t-1}}}
\newcommand{\z}[1]{\color{red}{z_{#1}^{t}}}
\newcommand{\r}[1]{\color{blue}{r_{#1}^{t}}}
% Explicit Vector Elements
\newcommand{\gfull}[2][]{\color{green}{g^{#1} \left( w^h_{#2} x_t + u^h_{#2 1} \r{1} \h{1} + u^h_{#2 2} \r{2} \h{2} \right)}}
\newcommand{\zfull}[2][]{\color{red}{\sigma^{#1} \left(w^z_{#2} x_t + u^z_{#2 1}h_1^{t-1}+u^z_{#2 2} h_2^{t-1}\right)}}
\newcommand{\rfull}[2][]{\color{blue}{\sigma^{#1} \left(w^r_{#2} x_t + u^r_{#2 1}h_1^{t-1}+u^r_{#2 2} h_2^{t-1}\right)}}
\newcommand{\gfullfull}[2][]{\color{green}{g^{#1} \left( w^h_{#2} x_t + u^h_{#2 1} \rfull{1} \h{1} + u^h_{#2 2} \rfull{2} \h{2} \right)}}
% Weight Matrices
\newcommand{\Wh}{\color{green}{W_h}}
\newcommand{\Wz}{\color{red}{W_z}}
\newcommand{\Wr}{\color{blue}{W_r}}
\newcommand{\Uh}{\color{green}{U_h}}
\newcommand{\Uz}{\color{red}{U_z}}
\newcommand{\Ur}{\color{blue}{U_r}}
% Weight Matrix Elements
\newcommand{\wh}[1]{\color{green}{w^h_{#1}}}
\newcommand{\wz}[1]{\color{red}{w^z_{#1}}}
\newcommand{\wr}[1]{\color{blue}{w^r_{#1}}}
\newcommand{\uh}[2]{\color{green}{u^h_{#1 #2}}}
\newcommand{\uz}[2]{\color{red}{u^z_{#1 #2}}}
\newcommand{\ur}[2]{\color{blue}{u^r_{#1 #2}}}
% Miscellaneous
\newcommand{\dxt}[1]{\color{#1}{x_t}}
\newcommand{\yhat}{\hat{y}_t}
\newcommand{\deriv}[2]{\frac{\partial #1}{\partial #2}}
\DeclareMathOperator{\diag}{diag}
$
Consider again, our low-dimensional numerical example with only one case with features $x = \begin{pmatrix}x_1, & x_2, & x_3 \end{pmatrix}^T$ and label $y$. Additionally let's again assume we only have two neurons in our network. This means for the weight matrices 

\begin{alignat}{2}
\color{green}{W_h} &= \begin{bmatrix} \wh{1} \\ \wh{2} \end{bmatrix}  \quad
\color{green}{U_h} &&= \begin{bmatrix} \uh{1}{1} & \uh{1}{2} \\ \uh{2}{1} & \uh{2}{2}\end{bmatrix}\\ 
\color{red}{W_z} &= \begin{bmatrix} \wz{1} \\ \wz{2} \end{bmatrix}  \quad
\color{red}{U_z} &&= \begin{bmatrix} \uz{1}{1} & \uz{1}{2} \\ \uz{2}{1} & \uz{2}{2} \end{bmatrix}\\
\color{blue}{W_r} &= \begin{bmatrix} \wr{1} \\ \wr{2} \end{bmatrix} \quad
\color{blue}{U_r} &&= \begin{bmatrix} \ur{1}{1} & \ur{1}{2} \\ \ur{2}{1} & \ur{2}{2} \end{bmatrix} 
\end{alignat}

We define the potential candidate update as
\begin{align}
\Htil &= \Gfull \\ &= \begin{pmatrix} \gfull{1} \\ \gfull{2} \end{pmatrix} \\ &= \begin{pmatrix} \gfullfull{1} \\ \gfullfull{2} \end{pmatrix},
\end{align}
the gates as
\begin{align}
\Z = \Zfull &= \begin{pmatrix} \z{1} \\ \z{2} \end{pmatrix} = \begin{pmatrix} \zfull{1} \\ \zfull{2} \end{pmatrix}  \quad\text{and}\\
\R = \Rfull &= \begin{pmatrix} \r{1} \\ \r{2} \end{pmatrix} = \begin{pmatrix} \rfull{1} \\ \rfull{2} \end{pmatrix},
\end{align}
where $g$ is the $tanh$-function and $\sigma$ the sigmoid function.

Finally, we define the update of the hidden state as
\begin{align}
h_t &= \color{red}{1-\Z} * h_{t-1} + \Z * \Htil \\ 
    &= 
\begin{pmatrix}
\color{red}{(1-\z{1})} h^{t-1}_1 + \z{1} \gfull{1} \\
\color{red}{(1-\z{2})} h^{t-1}_2 + \z{2} \gfull{2}
\end{pmatrix} \\
    &= 
\begin{pmatrix}
\color{red}{(1-\zfull{1})} h^{t-1}_1 + \zfull{1} \gfullfull{1} \\
\color{red}{(1-\zfull{2})} h^{t-1}_2 + \zfull{2} \gfullfull{2}
\end{pmatrix}
\end{align} \[[cp Rey, Salem, 2017](https://arxiv.org/pdf/1701.05923.pdf)\].

For the actual prediction at a step $t$, we connect every output of our hidden state in a dense layer. Which means that we are taking a weighted sum of all two of them.

\begin{align}
\hat{y}_t &= W^T h_t = w_1 h^t_1 + w_2 h^t_2 \\ &= w_1 \color{red}{(1-\zfull{1})} h^{t-1}_1 \\&\quad + \zfull{1} \gfullfull{1} \\ &+ w_2
\color{red}{(1-\zfull{2})} h^{t-1}_2 \\&\quad+ \zfull{2} \gfullfull{2},
\end{align}

Where $W = \begin{pmatrix} w_1 \\ w_2 \end{pmatrix}$ is the matrix that contains the weights for the outer layer. In our case this is just a $2\times 1$ vector.

### Weights for the candidate $\tilde{h}_t$

Let's start with taking partial derivatives of $\hat{y}_t$ with respect to the elemens of $\Wh$:

\begin{equation}
\frac{\partial \hat{y}_t}{\partial \wh{1}} = w_1 \z{1} \gfull[\prime]{1} \color{green}{x_t}
\end{equation}

Which is just an application of the chain-rule for derivatives. Similarly we get

\begin{equation}
\frac{\partial \hat{y}_t}{\partial \wh{2}} = w_2 \z{2} \gfull[\prime]{2} \color{green}{x_t}.
\end{equation}

We can obtain the gradient by combining the two partial derivatives in the following manner

\begin{align}
\frac{\partial \hat{y}_t}{\partial \Wh} &= 
    \begin{bmatrix} 
        \frac{\partial \hat{y}_t}{\partial \wh{1}} \\ \frac{\partial \hat{y}_t}{\partial \wh{2}}  
    \end{bmatrix}                       =
    \begin{bmatrix} 
        w_1 \z{1} \gfull{1} \color{green}{x_t} \\
        w_2 \z{2} \gfull{2} \color{green}{x_t}
    \end{bmatrix} \\
                                        &= W * \Z * \Gfull[\prime] \color{green}{x_t}.
\end{align}

Additionally, we can make use of the element-wise product in order to obtain a formulation that has a straightforward `numpy` implementation. We get the same result if we deploy the logic of the backpropagation algorithm.

\begin{align}
\frac{\partial {loss}_t}{\partial W_h} &= \frac{\partial {loss}_t}{\partial \hat{y}_t}\frac{\partial \hat{y}_t}{\partial h_t}\frac{\partial h_t}{\partial \tilde{h}_t}\frac{\partial \tilde{h}_t}{\partial \Wh} \\
                                   &= -(y-\hat{y}_t) W * \Z * \Gfull[\prime] \color{green}{x_t}
\end{align}

We continue with the partial derivatives of $\hat{y}_t$ with respect to the elements of $\Uh$

\begin{align}
\frac{\partial \hat{y}_t}{\partial \uh{1}{1}} &= w_1 \z{1} \gfull[\prime]{1} \r{1} \h{1} \\
\frac{\partial \hat{y}_t}{\partial \uh{1}{2}} &= w_1 \z{1} \gfull[\prime]{1} \r{2} \h{2} \\
\frac{\partial \hat{y}_t}{\partial \uh{2}{1}} &= w_2 \z{2} \gfull[\prime]{2} \r{1} \h{1} \\
\frac{\partial \hat{y}_t}{\partial \uh{2}{2}} &= w_2 \z{2} \gfull[\prime]{2} \r{2} \h{2}.
\end{align}

These we can combine to form the jacobian of of $\yhat$ with respect to $\Uh$

\begin{align}
\deriv{\yhat}{\Uh} &= 
\begin{bmatrix} 
    \deriv{\yhat}{\uh{1}{1}} & \deriv{\yhat}{\uh{1}{2}} \\
    \deriv{\yhat}{\uh{2}{1}} & \deriv{\yhat}{\uh{2}{2}}
\end{bmatrix}      = 
\begin{bmatrix}
w_1 \z{1} \gfull[\prime]{1} \r{1} \h{1} &
w_1 \z{1} \gfull[\prime]{1} \r{2} \h{2} \\
w_2 \z{2} \gfull[\prime]{2} \r{1} \h{1} &
w_2 \z{2} \gfull[\prime]{2} \r{2} \h{2}.
\end{bmatrix} \\
                  &=
\begin{bmatrix} w_1 & w_1 \\ w_2 & w_2 \end{bmatrix} * 
\begin{bmatrix} \z{1} & \z{1} \\ \z{2} & \z{2} \end{bmatrix} * 
\begin{bmatrix} \gfull[\prime]{1} & \gfull[\prime]{1} \\ \gfull[\prime]{2} & \gfull[\prime]{2} \end{bmatrix} * 
\begin{bmatrix} \r{1} & \r{2} \\ \r{1} & \r{2} \end{bmatrix} * 
\begin{bmatrix} \h{1} & \h{2} \\ \h{1} & \h{2} \end{bmatrix} \\
                  &=
\begin{bmatrix} w & w \end{bmatrix} * \begin{bmatrix} \Z & \Z \end{bmatrix} * \begin{bmatrix} \Gfull[\prime] & \Gfull[\prime] \end{bmatrix} * \begin{bmatrix} \R^T \\ \R^T \end{bmatrix} * \begin{bmatrix} \H^T \\ \H^T \end{bmatrix} \\
&= \diag(w) \cdot \diag(\Z) \cdot \diag(\Gfull[\prime]) \cdot \mathbb{1} \mathbb{1}^T \cdot \diag(\R) \cdot \diag(\H)
\end{align}

Again, following the logic of the backpropagation algorithm, we obtain the same result

\begin{align}
\frac{\partial {loss}_t}{\partial \Uh} &= \frac{\partial {loss}_t}{\partial \hat{y}_t}\frac{\partial \hat{y}_t}{\partial h_t}\frac{\partial h_t}{\partial \tilde{h}_t}\frac{\partial \tilde{h}_t}{\partial \Uh} \\
                                    &= -(y-\hat{y}_t)\ \diag \left[ w * \Z * \Gfull[\prime]\right] \cdot  \begin{bmatrix} (\R * \H)^T \\ (\R * \H)^T \end{bmatrix}
\end{align}

#### Weights for the update gate $z_t$

Using the same logic, we calculate the partial derivatives of $\yhat$ with respect to the elements of $\Wz$, which gives 

\begin{equation}
\deriv{\yhat}{\wz{1}} = w_1 \left(\color{red}{-} \zfull[\prime]{1}\right) \h{1} + \htil{1} \zfull[\prime]{1} \dxt{red},
\end{equation}


\begin{equation}
\deriv{\yhat}{\wz{2}} = w_2 \left(\color{red}{-} \zfull[\prime]{2}\right) \h{1} + \htil{1} \zfull[\prime]{2} \dxt{red},
\end{equation}

and when combined leaves us the gradient of $\yhat$ with respect to $\Wz$ as

\begin{align}
\deriv{\yhat}{\Wz} &= \begin{bmatrix}
\deriv{\yhat}{\wz{1}} \\ \deriv{\yhat}{\wz{2}}
\end{bmatrix} = 
\begin{bmatrix}
    w_1 \left(\color{red}{-} \zfull[\prime]{1}\right) \h{1} + \htil{1} \zfull[\prime]{1} \dxt{red} \\
    w_2 \left(\color{red}{-} \zfull[\prime]{2}\right) \h{1} + \htil{1} \zfull[\prime]{2} \dxt{red}
\end{bmatrix} \\
&= w * \begin{bmatrix} -\H + \Htil \end{bmatrix} * \Zfull[\prime] \dxt{red}
\end{align}


By now this should be getting old. We take partial derivatives of $\yhat$ with respect to each element of $\Uz$. This leaves us with

\begin{align}
\deriv{\yhat}{\uz{1}{1}} &= w_1 \left[\color{red}{(- \zfull[\prime]{1} \h{1} )} \h{1} + \htil{1} \zfull[\prime]{1} \h{1} \right], \\
\deriv{\yhat}{\uz{1}{2}} &= w_1 \left[\color{red}{(- \zfull[\prime]{1} \h{2} )} \h{1} + \htil{1} \zfull[\prime]{1} \h{2} \right], \\
\deriv{\yhat}{\uz{2}{1}} &= w_2 \left[\color{red}{(- \zfull[\prime]{2} \h{1} )} \h{2} + \htil{2} \zfull[\prime]{2} \h{1} \right] \text{and} \\
\deriv{\yhat}{\uz{2}{2}} &= w_2 \left[\color{red}{(- \zfull[\prime]{2} \h{2} )} \h{2} + \htil{2} \zfull[\prime]{2} \h{2} \right]. 
\end{align}

We can combine these partial derivatives in order to form the jacobian of $\yhat$ with respect to $\Uz$ as

\begin{align}
\deriv{\yhat}{\Uz} &= 
\begin{bmatrix} 
    \deriv{\yhat}{\uz{1}{1}} & \deriv{\yhat}{\uz{1}{2}} \\
    \deriv{\yhat}{\uz{2}{1}} & \deriv{\yhat}{\uz{2}{2}}
\end{bmatrix} \\ &=
\begin{bmatrix}  
     w_1 \left[\color{red}{(- \zfull[\prime]{1} \h{1} )} \h{1} + \htil{1} \zfull[\prime]{1} \h{1} \right] &
     w_1 \left[\color{red}{(- \zfull[\prime]{1} \h{2} )} \h{1} + \htil{1} \zfull[\prime]{1} \h{2} \right] \\
     w_2 \left[\color{red}{(- \zfull[\prime]{2} \h{1} )} \h{2} + \htil{2} \zfull[\prime]{2} \h{1} \right] &
     w_2 \left[\color{red}{(- \zfull[\prime]{2} \h{2} )} \h{2} + \htil{2} \zfull[\prime]{2} \h{2} \right] 
\end{bmatrix} \\ &=
\begin{bmatrix} w & w \end{bmatrix} * 
\begin{bmatrix} \Htil - \H & \Htil - \H \end{bmatrix} * 
\begin{bmatrix} \Zfull[\prime] & \Zfull[\prime] \end{bmatrix} * 
\begin{bmatrix} \H^T \\ \H^T \end{bmatrix} \\
&= \diag(w) \cdot \diag(\Htil - \H) \cdot \Zfull[\prime] \cdot \H^T.
\end{align}

And by the same token, we need to use the previous derivations in order to find the derivative of the loss function with respect to $\Uz$, where now one can easily see that:

\begin{align}
\frac{\partial {loss}_t}{\partial \Uz} &= \frac{\partial loss}{\partial \hat{y}_t}\frac{\partial \hat{y}_t}{\partial h_t}\frac{\partial h_t}{\partial z_t}\frac{\partial z_t}{\partial \Uz} \\
                                   &= -(y-\hat{y}_t)\ \diag \left[ w * [-\H + \Htil] * \Zfull[\prime]\right] \cdot \begin{bmatrix} \H^T \\ \H^T \end{bmatrix}.
\end{align}

Having done that, we can implement it in the now too familiar way.

#### Weights for the reset gate $r_t$

Similarly, we can proceed with obtaining the gradients of $\yhat$ with respect to the elements of the reset gate.

\begin{align}
\deriv{\yhat}{\wr{1}} &= w_1 \z{1} \gfull[\prime]{1} \uh{1}{1} \h{1} \rfull[\prime]{1} \dxt{blue} \\ 
&\quad + w_2 \z{2} \gfull[\prime]{2} \uh{2}{1} \h{1} \rfull[\prime]{1} \dxt{blue}
\end{align}

and

\begin{align}
\deriv{\yhat}{\wr{2}} &= w_1 \z{1} \gfull[\prime]{1} \uh{1}{2} \h{2} \rfull[\prime]{2} \dxt{blue} \\ 
&\quad + w_2 \z{2} \gfull[\prime]{2} \uh{2}{2} \h{2} \rfull[\prime]{2} \dxt{blue}.
\end{align}

By collecting the partial derivatives into a vector we form the gradient of $\yhat$ with respect to $\Wr$

\begin{align}
\deriv{\yhat}{\Wr} &= \begin{bmatrix}\deriv{\yhat}{\wr{1}} \\ \deriv{\yhat}{\wr{2}} \end{bmatrix} \\ 
&= w^T \cdot \left(
\begin{bmatrix} \Z & \Z \end{bmatrix} * 
\begin{bmatrix} \Gfull[\prime] & \Gfull[\prime] \end{bmatrix} * 
\Uh *
\begin{bmatrix} \H^T \\ \H^T \end{bmatrix} *
\begin{bmatrix} \Rfull[\prime]^T \\ \Rfull[\prime]^T \end{bmatrix}
\right) \dxt{blue} \\
&= w^T \cdot \diag(\Z) \cdot \diag(\Gfull[\prime]) \cdot \Uh \cdot \diag(\H)\cdot \diag(\Rfull[\prime]) \dxt{blue}.
\end{align}

Finally, we compute the partial derivatives of $\yhat$ with respect to $\Ur$

\begin{align}
\deriv{\yhat}{\ur{1}{1}} &= w_1 \z{1} \gfull[\prime]{1} \uh{1}{1} \h{1} \rfull[\prime]{1} \h{1} \\ 
                         & \quad + w_2 \z{2} \gfull[\prime]{2} \uh{2}{1} \h{1} \rfull[\prime]{1} \h{1} \\
\deriv{\yhat}{\ur{1}{2}} &= w_1 \z{1} \gfull[\prime]{1} \uh{1}{1} \h{1} \rfull[\prime]{1} \h{2} \\
                         & \quad + w_2 \z{2} \gfull[\prime]{2} \uh{2}{1} \h{1} \rfull[\prime]{1} \h{2} \\
\deriv{\yhat}{\ur{2}{1}} &= w_1 \z{1} \gfull[\prime]{1} \uh{1}{2} \h{2} \rfull[\prime]{2} \h{1} \\
                         & \quad + w_2 \z{2} \gfull[\prime]{2} \uh{2}{2} \h{2} \rfull[\prime]{2} \h{1} \\
\deriv{\yhat}{\ur{2}{2}} &= w_1 \z{1} \gfull[\prime]{1} \uh{1}{2} \h{2} \rfull[\prime]{2} \h{2} \\
                         & \quad + w_2 \z{2} \gfull[\prime]{2} \uh{2}{2} \h{2} \rfull[\prime]{2} \h{2}.
\end{align}

And by combining all partial derivatives, we obtain the jacobian of $\yhat$ with respect to $\Ur$

\begin{align}
\deriv{\yhat}{\Ur} &=
\begin{bmatrix} 
    \deriv{\yhat}{\ur{1}{1}} & \deriv{\yhat}{\ur{1}{2}} \\
    \deriv{\yhat}{\ur{2}{1}} & \deriv{\yhat}{\ur{2}{2}}
\end{bmatrix} \\ &= 
\begin{bmatrix}(w * \Z * \Gfull[\prime])^T \\ (w * \Z * \Gfull[\prime])^T \end{bmatrix} \cdot \Uh^T  * 
\begin{bmatrix} \H \cdot \H^T \end{bmatrix} *
\begin{bmatrix} \Rfull[\prime] & \Rfull[\prime] \end{bmatrix} \\ &=
\diag(w) \cdot \diag(\Z) \cdot \diag(\Gfull[\prime]) \cdot \Uh^T \cdot \diag(\H) \cdot \diag(\Rfull[\prime]) \cdot \mathbb{1}\mathbb{1}^T \cdot \diag(\H).
\end{align}.